<a href="https://colab.research.google.com/github/taofick221/Brain-tumor-detection-CNN/blob/main/94validation(model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocess code


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from PIL import Image
import numpy as np

# Paths
input_base_path = '/content/drive/MyDrive/Thesis/dataset2.0'
output_base_path = '/content/drive/MyDrive/Thesis/preprocess_data3.0'


# Image size and augmentation settings
IMG_SIZE = (224, 224)
AUG_PER_IMAGE = 5

datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create output folders
for split in ['Training', 'Testing']:
    split_input = os.path.join(input_base_path, split)
    split_output = os.path.join(output_base_path, split)

    for class_name in os.listdir(split_input):
        input_folder = os.path.join(split_input, class_name)
        output_folder = os.path.join(split_output, class_name)
        os.makedirs(output_folder, exist_ok=True)

        print(f"📂 Processing Class: {split}/{class_name}")

        for img_name in os.listdir(input_folder):
            img_path = os.path.join(input_folder, img_name)

            try:
                img = load_img(img_path, target_size=IMG_SIZE)
                x = img_to_array(img)
                x = x.reshape((1,) + x.shape)

                # Save original image as well (resized)
                img.save(os.path.join(output_folder, img_name))

                # Generate augmented images
                i = 0
                for batch in datagen.flow(
                        x,
                        batch_size=1,
                        save_to_dir=output_folder,
                        save_prefix=f'{class_name}_aug',
                        save_format='jpg'):
                    i += 1
                    if i >= AUG_PER_IMAGE:
                        break

            except Exception as e:
                print(f"❌ Skipping {img_name} due to error: {e}")

print("✅ All preprocessing & augmentation completed successfully!")


📂 Processing Class: Training/notumor
📂 Processing Class: Training/meningioma
📂 Processing Class: Training/pituitary
📂 Processing Class: Training/glioma
📂 Processing Class: Testing/pituitary
📂 Processing Class: Testing/notumor
📂 Processing Class: Testing/meningioma
📂 Processing Class: Testing/glioma
✅ All preprocessing & augmentation completed successfully!


# Model training


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

# === CONFIG ===
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 4

# === PATHS ===
train_path = '/content/drive/MyDrive/Thesis/preprocess_data3.0/Training'
val_path = '/content/drive/MyDrive/Thesis/preprocess_data3.0/Testing'
model_save_path = '/content/drive/MyDrive/Thesis/final_model/best_custom_model.keras'

# === DATA LOADING ===
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# === MODEL ===
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Prevent overfitting
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# === CALLBACKS ===
checkpoint = ModelCheckpoint(
    model_save_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

lr_schedule = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=1
)

# === TRAIN MODEL ===
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop, lr_schedule]
)


Found 26151 images belonging to 4 classes.
Found 7340 images belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
818/818 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.6410 - loss: 0.8386
Epoch 1: val_accuracy improved from -inf to 0.75095, saving model to /content/drive/MyDrive/Thesis/final_model/best_custom_model.keras
818/818 ━━━━━━━━━━━━━━━━━━━━ 9636s 12s/step - accuracy: 0.6411 - loss: 0.8384 - val_accuracy: 0.7510 - val_loss: 0.6093 - learning_rate: 5.0000e-04
Epoch 2/20
818/818 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8260 - loss: 0.4422
Epoch 2: val_accuracy improved from 0.75095 to 0.82262, saving model to /content/drive/MyDrive/Thesis/final_model/best_custom_model.keras
818/818 ━━━━━━━━━━━━━━━━━━━━ 3510s 4s/step - accuracy: 0.8261 - loss: 0.4421 - val_accuracy: 0.8226 - val_loss: 0.4516 - learning_rate: 5.0000e-04
Epoch 3/20
818/818 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8821 - loss: 0.2985
Epoch 3: val_accuracy improved from 0.82262 to 0.87834, saving model to /content/drive/MyDrive/Thesis/final_model/best_custom_model.keras
818/818 ━━━━━━━━━━━━━━━━━━━━ 3014s 4s/s